# Подготовка

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [46]:
from pyspark import SparkContext, SparkConf
from typing import NamedTuple
from datetime import datetime
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql import functions as func
from pyspark.sql.window import Window
import numpy as np
import re

In [5]:
spark = SparkSession.builder\
        .master("local[*]")\
        .appName("LR2")\
        .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.14.0")\
        .getOrCreate()

In [58]:
prog_language_data = spark.read\
.option("header", True)\
.option("inferSchema", True)\
.option("timestampFormat", 'M/d/y H:m')\
.csv("programming-languages.csv")

In [59]:
prog_language_data.printSchema()

root
 |-- name: string (nullable = true)
 |-- wikipedia_url: string (nullable = true)



In [60]:
prog_language_data.show(n=5, truncate=False)

+----------+---------------------------------------------------------+
|name      |wikipedia_url                                            |
+----------+---------------------------------------------------------+
|A# .NET   |https://en.wikipedia.org/wiki/A_Sharp_(.NET)             |
|A# (Axiom)|https://en.wikipedia.org/wiki/A_Sharp_(Axiom)            |
|A-0 System|https://en.wikipedia.org/wiki/A-0_System                 |
|A+        |https://en.wikipedia.org/wiki/A%2B_(programming_language)|
|A++       |https://en.wikipedia.org/wiki/A%2B%2B                    |
+----------+---------------------------------------------------------+
only showing top 5 rows



In [62]:
posts_path = '/content/posts_sample.xml'

In [63]:
posts_data = spark.read.format('xml').options(rowTag='row').load(posts_path)

In [64]:
posts_data.printSchema()

root
 |-- _AcceptedAnswerId: long (nullable = true)
 |-- _AnswerCount: long (nullable = true)
 |-- _Body: string (nullable = true)
 |-- _ClosedDate: timestamp (nullable = true)
 |-- _CommentCount: long (nullable = true)
 |-- _CommunityOwnedDate: timestamp (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _FavoriteCount: long (nullable = true)
 |-- _Id: long (nullable = true)
 |-- _LastActivityDate: timestamp (nullable = true)
 |-- _LastEditDate: timestamp (nullable = true)
 |-- _LastEditorDisplayName: string (nullable = true)
 |-- _LastEditorUserId: long (nullable = true)
 |-- _OwnerDisplayName: string (nullable = true)
 |-- _OwnerUserId: long (nullable = true)
 |-- _ParentId: long (nullable = true)
 |-- _PostTypeId: long (nullable = true)
 |-- _Score: long (nullable = true)
 |-- _Tags: string (nullable = true)
 |-- _Title: string (nullable = true)
 |-- _ViewCount: long (nullable = true)



In [65]:
posts_data.show(n=5, truncate=False)

+-----------------+------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+-------------+-----------------------+-----------------------+--------------+---+-----------------------+-----------------------+----------------------+-----------------+-----------------+------------+---------+-----------+------+------------------------------------------------------+-----

# Задание

In [97]:
# получение списка всех языков программирования
prog_language_list = prog_language_data.select("name").collect()
prog_language_list = [lang.name.lower() for lang in prog_language_list]
prog_language_list

['a# .net',
 'a# (axiom)',
 'a-0 system',
 'a+',
 'a++',
 'abap',
 'abc',
 'abc algol',
 'abset',
 'absys',
 'acc',
 'accent',
 'ace dasl',
 'acl2',
 'act-iii',
 'action!',
 'actionscript',
 'ada',
 'adenine',
 'agda',
 'agilent vee',
 'agora',
 'aimms',
 'alef',
 'alf',
 'algol 58',
 'algol 60',
 'algol 68',
 'algol w',
 'alice',
 'alma-0',
 'ambienttalk',
 'amiga e',
 'amos',
 'ampl',
 'apex (salesforce.com)',
 'apl',
 "app inventor for android's visual block language",
 'applescript',
 'arc',
 'arexx',
 'argus',
 'aspectj',
 'assembly language',
 'ats',
 'ateji px',
 'autohotkey',
 'autocoder',
 'autoit',
 'autolisp / visual lisp',
 'averest',
 'awk',
 'axum',
 'b',
 'babbage',
 'bash',
 'basic',
 'bc',
 'bcpl',
 'beanshell',
 'batch (windows/dos)',
 'bertrand',
 'beta',
 'bigwig',
 'bistro',
 'bitc',
 'bliss',
 'blockly',
 'bloop',
 'blue',
 'boo',
 'boomerang',
 'bourne shell (including',
 'bash and',
 'ksh )',
 'brew',
 'bpel',
 'c',
 'c--',
 'c++ – iso/iec 14882',
 'c# – iso/iec

In [67]:
# извлечение из тегов названий ЯП
def get_language_from_tag(tag):
  if tag is None:
    return []
  # поиск по регулярному выражению
  tags = re.findall(r'<([^<>]*)>', tag)
  langs = []
  for i in range(len(tags)):
    # сохраняем в список, если тег - ЯП
    if (tags[i] in prog_language_list):
      langs.append(tags[i])
  return langs

In [92]:
# udf функция
udf_get_lang = func.udf(get_language_from_tag, ArrayType(StringType()))

posts_data_langs = posts_data.withColumn("prog_langs", udf_get_lang(posts_data._Tags))\ # применение udf
                        .withColumn("year", func.year(posts_data._CreationDate))\ # извлечение года
                        .select("year", func.explode("prog_langs").alias("lang"), "_ViewCount") # выборка необходимых значений
# оставляем записи с 2010 по 2020 год
posts_data_langs_period = posts_data_langs.filter((posts_data_langs.year >= 2010) & (posts_data_langs.year <= 2020))
posts_data_langs_period.show(10)

+----+-----------+----------+
|year|       lang|_ViewCount|
+----+-----------+----------+
|2010|       java|       132|
|2010|        php|      1258|
|2010|       ruby|      9649|
|2010|          c|      2384|
|2010|        php|      1987|
|2010|     python|      3321|
|2010| javascript|       128|
|2010|applescript|       477|
|2010|        php|      1748|
|2010|        php|       998|
+----+-----------+----------+
only showing top 10 rows



In [93]:
# получение списка ЯП суммарным просмотрам в году - популярности ЯП
top_langs = posts_data_langs_by_year.groupBy("year", "lang").agg({"_ViewCount": "sum"})\
            .withColumnRenamed("sum(_ViewCount)", "popularity")
top_langs.show(10)

+----+-----------+----------+
|year|       lang|popularity|
+----+-----------+----------+
|2019| typescript|      1577|
|2017|       perl|      2366|
|2012|       bash|      9333|
|2011|objective-c|    218934|
|2013|        php|    471986|
|2018|    jscript|        38|
|2013|       chef|     62421|
|2017|       curl|       897|
|2013|     delphi|     20143|
|2016|        awk|      1808|
+----+-----------+----------+
only showing top 10 rows



In [100]:
# оконная функция по популярности
window_func = Window.partitionBy('year').orderBy(func.desc('popularity'))
# получение ранга ЯП по году
langs_rank = top_langs.withColumn('rank', func.row_number().over(window_func))
# оставляем только 10 позиций
top_languages_per_year = langs_rank.filter(langs_rank['rank'] <= 10)

In [102]:
# cохранение результата в формате Apache Parquet
top_languages_per_year.write.mode('overwrite').parquet('top_languages_per_year.parquet')

In [101]:
# чтение результата из файла parquet
top_languages_parquet = spark.read.parquet('top_languages_per_year.parquet')

top_languages_parquet.show(n=100)

+----+-----------+----------+----+
|year|       lang|popularity|rank|
+----+-----------+----------+----+
|2010|        php|   1189629|   1|
|2010|       java|    563211|   2|
|2010| javascript|    316131|   3|
|2010|objective-c|     97009|   4|
|2010|       ruby|     76215|   5|
|2010|          c|     66587|   6|
|2010|     python|     60672|   7|
|2010|     matlab|     51865|   8|
|2010|applescript|     32305|   9|
|2010|     delphi|     13065|  10|
|2011| javascript|    809078|   1|
|2011|       java|    389834|   2|
|2011|        php|    246770|   3|
|2011|          c|    238277|   4|
|2011|objective-c|    218934|   5|
|2011|     python|    203180|   6|
|2011|       bash|     60805|   7|
|2011|       ruby|     39223|   8|
|2011|       perl|     28502|   9|
|2011|     matlab|     18816|  10|
|2012|       java|    661770|   1|
|2012| javascript|    572750|   2|
|2012|        php|    459506|   3|
|2012|     python|    275337|   4|
|2012|       ruby|    104610|   5|
|2012|objective-c|  